# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298774730998                   -0.85    5.1
  2   -8.300236283950       -2.84       -1.26    1.2
  3   -8.300447034184       -3.68       -1.89    3.2
  4   -8.300461495618       -4.84       -2.76    1.4
  5   -8.300464146807       -5.58       -3.06    6.2
  6   -8.300464386491       -6.62       -3.24    1.0
  7   -8.300464514004       -6.89       -3.38    8.9
  8   -8.300464581354       -7.17       -3.51    1.2
  9   -8.300464629485       -7.32       -3.71    1.0
 10   -8.300464638835       -8.03       -3.88    1.0
 11   -8.300464643663       -8.32       -4.21    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67478530848                   -0.70    6.8
  2   -16.67870406422       -2.41       -1.14    1.0
  3   -16.67922182407       -3.29       -1.86    5.2
  4   -16.67928113371       -4.23       -2.75    2.0
  5   -16.67928604663       -5.31       -3.19    4.8
  6   -16.67928621473       -6.77       -3.50    3.5
  7   -16.67928622168       -8.16       -3.95    1.0
  8   -16.67928622406       -8.62       -4.59    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32518944027                   -0.56    7.5
  2   -33.33413100719       -2.05       -1.00    1.8
  3   -33.33600974775       -2.73       -1.73    4.9
  4   -33.33661932125       -3.21       -2.64    5.2
  5   -33.33694217592       -3.49       -3.09   10.0
  6   -33.33694370369       -5.82       -3.61    5.0
  7   -33.33694392175       -6.66       -4.26    3.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298535579976                   -0.85    5.1
  2   -8.300281313723       -2.76       -1.59    1.2
  3   -8.300454530164       -3.76       -2.60    4.6
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       3
│       3
│       3
│       3
│       3
│       3
│       2
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:34
  4   -8.300399919524   +   -4.26       -2.35   15.0
  5   -8.300464427755       -4.19       -3.62    4.4
  6   -8.300464591800       -6.79       -3.95    2.6
  7   -8.300464643652       -7.29       -5.00    2.0


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30482259344                   -0.56    7.6
  2   -33.30411558740   +   -3.15       -1.27    1.0
  3   -22.61242708042   +    1.03       -0.55   10.1
  4   -33.23444314310        1.03       -1.42    7.2
  5   -33.27813884759       -1.36       -1.44    3.1
  6   -32.48881563719   +   -0.10       -1.10    4.1
  7   -33.25485212464       -0.12       -1.60    4.8
  8   -33.33237139215       -1.11       -2.15    3.9
  9   -33.33627868794       -2.41       -2.40    3.0
 10   -33.33636311819       -4.07       -2.48    2.2
 11   -33.33680285590       -3.36       -2.94    2.0
 12   -33.33694062190       -3.86       -3.31    3.2
 13   -33.33694115875       -6.27       -3.54    3.5
 14   -33.33694322039       -5.69       -3.74    1.6
 15   -33.33692344304   +   -4.70       -3.41    4.0
 16   -33.33694348745       -4.70       -4.18    3.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.